In [1]:
# Specify device
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

In [ ]:
import importlib
import cv2
import json
import os
import sys

# change this property
NOMEROFF_NET_DIR = os.path.abspath('/var/www/nomeroff/nomeroff-net')
sys.path.append(NOMEROFF_NET_DIR)

In [2]:
def compare(model_path="./models/anpr_ocr_su_2020_11_25_tensorflow_v2.3.h5", 
           text_detector_name = "su",
           img_format = "png",
           root_dir='./autoriaNumberplateOcrSu-2020-11-26/test',
           predicted_part_size=300,
           acc_less_than = 0.7,
           replace_tamplate = {'moderation': {'isModerated': 1, 'moderatedBy': 'ApelSYN'}}):
    text_detector_module = importlib.import_module("NomeroffNet.TextDetectors."+text_detector_name)
    text_detector = getattr(text_detector_module, text_detector_name)()
    text_detector.load(model_path)

    ann_dir = os.path.join(root_dir, "ann")
    jsons = []
    jsons_paths = []
    for dirName, subdirList, fileList in os.walk(ann_dir):
        for fname in fileList:
            fname = os.path.join(ann_dir, fname)
            jsons_paths.append(fname)
            with open(fname) as jsonF:
                jsonData = json.load(jsonF)
            jsons.append(jsonData)
    print("LOADED {} ANNOTATIONS".format(len(jsons)))

    img_dir = os.path.join(root_dir, "img")
    imgs = []                
    for j in jsons:
        img_path =os.path.join(img_dir, "{}.{}".format(j["name"], img_format))
        img = cv2.imread(img_path)
        imgs.append(img)
    print("LOADED {} IMAGES".format(len(imgs)))

    predicted = []
    accs      = []
    N = int(len(imgs) / predicted_part_size) + 1
    for i in range(N):
        part           = i*predicted_part_size
        decoded        = [jsonData["moderation"]["predicted"] for jsonData in jsons[part:part+predicted_part_size]]
        part_imgs      = imgs[part:part+predicted_part_size]
        predicted_part, net_out_value_part = text_detector.predict(part_imgs, return_acc=True)
        predicted     += predicted_part
        
        
        # get accuracy
        if acc_less_than >= 1:
            # not process acc
            accs  += [1 for _predicted in predicted_part]
            continue
        # process accuracy
        acc_part = []
        for _predicted, _net_out_value in zip(predicted_part, net_out_value_part):
            acc_part.append(text_detector.get_acc([_net_out_value], [_predicted]))
        accs  += acc_part
        

    print("PREDICTED {} IMAGES".format(len(predicted)))

    err_cnt = 0
    for i in range(len(jsons_paths)):
        json_path      = jsons_paths[i]
        predicted_item = predicted[i]
        jsonData       = jsons[i]
        acc            = accs[i]
        jsonData["moderation"]["predicted"] = predicted_item
        
        #print(jsonData["description"],  jsonData["moderation"]["predicted"])
        if jsonData["description"] == jsonData["moderation"]["predicted"] and acc > acc_less_than :
            #jsonData.update(replace_tamplate)
            jsonData["moderation"]["isModerated"] = 1
        else:
            print("Predicted '{}' with acc {}, real: '{}' in file {}".format(
                jsonData["moderation"]["predicted"], 
                acc,
                jsonData["description"], 
                json_path))
            err_cnt = err_cnt+1

        jsonData["description"] = predicted_item
            
        with open(json_path, "w", encoding='utf8') as jsonWF:
            json.dump(jsonData, jsonWF,  ensure_ascii=False)

    print("Error detection count: {}".format(err_cnt))
    print("Accuracy: {}".format(1-err_cnt/len(predicted)))
    #print(accs)

In [1]:
compare()

NameError: name 'compare' is not defined